## Introduction

The GEE, or generalized estimating equation, is a model meant to estimate the parameters of a generalized linear model with unknown correlation between outcomes. They work with longitudinal data, multi-cohort data, and are chosen specifically to avoid the sensitivity to correlation structures a la GLMs.

### Necessary Libraries
We'll load the the necessary libraries into the environment and follow it up by importing the data.

In [3]:
library(RCurl)
library(gee)
library(tidyverse)

x <- getURL("https://raw.githubusercontent.com/ntellakula/GEEs/master/drug_use.csv")
drug_source <- read.csv(text = x)

head(drug_source)

Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang
Registered S3 method overwritten by 'rvest':
  method            from
  read_xml.response xml2
-- Attaching packages --------------------------------------- tidyverse 1.2.1 --
v ggplot2 3.1.1       v purrr   0.3.2  
v tibble  2.1.1       v dplyr   0.8.0.1
v tidyr   0.8.3       v stringr 1.4.0  
v readr   1.3.1       v forcats 0.4.0  
-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x tidyr::complete() masks RCurl::complete()
x dplyr::filter()   masks stats::filter()
x dplyr::lag()      masks stats::lag()


id,response,sex,race,type,count,i,CASE
<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,1,0,0,1,68,1,0
1,1,0,0,1,68,2,1
1,1,0,0,1,68,3,2
1,1,0,0,1,68,4,3
1,1,0,0,1,68,5,4
1,1,0,0,1,68,6,5


### Data Exploration

In [16]:
dim(drug_source)
drug_source %>% distinct(id)
drug_source %>% distinct(response)
drug_source %>% distinct(sex)
drug_source %>% distinct(race)
drug_source %>% distinct(type)
max(drug_source$i)
max(drug_source$CASE)

[1] 6828    8

id
<int>
1
2
3
4


response
<int>
1
0


sex
<int>
0
1


race
<int>
0
1


type
<int>
1
2
3


[1] 910

[1] 6827

We can observe that this is a 6828 by 8 data frame. The information about the data set isn't the clearest, but by looking at the unique values in each column, we can deduce a few things.

1. Type is the type of drug and there are 3 options. I also have the original data set so I have that omnipotent knowledge.
2. There are 2 races.
3. There are 2 sexes, probably M & F.

The last two variables are harder to deduce as they seem to count up and are continuous.

We now build a GEE. The first step is to relevel a few columns to set them as factors and declare reference levels.

In [18]:
drug_source$sex <- relevel(as.factor(drug_source$sex), ref = "0")
drug_source$type <- relevel(as.factor(drug_source$type), ref = "3")
head(drug_source)

id,response,sex,race,type,count,i,CASE
<int>,<int>,<fct>,<int>,<fct>,<int>,<int>,<int>
1,1,0,0,1,68,1,0
1,1,0,0,1,68,2,1
1,1,0,0,1,68,3,2
1,1,0,0,1,68,4,3
1,1,0,0,1,68,5,4
1,1,0,0,1,68,6,5


We observe that <code>sex</code> and <code>type</code> have now become factors. The next step is to create the actual model.

In [19]:
drug_model <- gee(formula = response ~ sex + race + type, id = CASE,
                  data = drug_source, family = binomial, corstr = "exchangeable")

Beginning Cgee S-function, @(#) geeformula.q 4.13 98/01/27
running glm to get initial regression estimate


(Intercept)        sex1        race       type1       type2 
-0.67139548 -0.04333689  0.40719846  2.10622159  0.96759336 


In [20]:
summary(drug_model)


 GEE:  GENERALIZED LINEAR MODELS FOR DEPENDENT DATA
 gee S-function, version 4.13 modified 98/01/27 (1998) 

Model:
 Link:                      Logit 
 Variance to Mean Relation: Binomial 
 Correlation Structure:     Exchangeable 

Call:
gee(formula = response ~ sex + race + type, id = CASE, data = drug_source, 
    family = binomial, corstr = "exchangeable")

Summary of Residuals:
       Min         1Q     Median         3Q        Max 
-0.8631880 -0.4237168  0.1420109  0.3407263  0.6714460 


Coefficients:
               Estimate Naive S.E.    Naive z Robust S.E.   Robust z
(Intercept) -0.67139548 0.10534864 -6.3730815  0.10618773 -6.3227217
sex1        -0.04333689 0.05456440 -0.7942337  0.05454414 -0.7945286
race         0.40719846 0.10013803  4.0663719  0.10084449  4.0378852
type1        2.10622159 0.07344012 28.6794410  0.07335092 28.7143183
type2        0.96759336 0.06135302 15.7709178  0.06137308 15.7657616

Estimated Scale Parameter:  1.000179
Number of Iterations:  1

Working 

### Model Interpretation

There's quite a bit to unpack here. The <code>family</code> that was selected is <code>binomial</code>. The dependent variables of the equation to calculate drug response are all binomial or categorical. Thus, whatever family permits use of the logit link function would be the most appropriate: <code>binomial</code>. In this type of model, we will observe odds and odds ratios, not the actual predicted values as we would with continuous predictors. With the binomial and logit link function, the predicted parameter and the predictors themselves all follow different distributions. The correlation structure is defined as <code>exchangeable</code>. This means we assume the observations within a subject are equally correlated. Furthermore, in this model, we are observing main effects only. We aren't looking at any interactions between predictors.

The equation of the model that can be used for predictive purposes would be:

$logit(\hat{\pi}) = −0.671 − 0.043S + 0.407R + 1.206T_1 + 0.968T_2$

Now, we want to find the parameter estimates, odds ratios, and p-values that will result in the described equation.

In [21]:
est <- data.frame(drug_model$coefficients)
est

,drug_model.coefficients
,<dbl>
(Intercept),-0.67139548
sex1,-0.04333689
race,0.40719846
type1,2.10622159
type2,0.96759336


In [25]:
stderr <- data.frame(sqrt(diag(drug_model$robust.variance)))
stderr

,sqrt.diag.drug_model.robust.variance..
,<dbl>
(Intercept),0.10618773
sex1,0.05454414
race,0.10084449
type1,0.07335092
type2,0.06137308


In [26]:
wald_z <- est/stderr
wald_z

,drug_model.coefficients
,<dbl>
(Intercept),-6.3227217
sex1,-0.7945286
race,4.0378852
type1,28.7143183
type2,15.7657616


In [27]:
p_values <- 2 * (1 - pnorm(abs(wald_z[ ,1])))
names(p_values) <- row.names(wald_z)
p_values

(Intercept)         sex1         race        type1        type2 
2.569960e-10 4.268877e-01 5.393523e-05 0.000000e+00 0.000000e+00

We can observe that sex does not have a significant p-value and ordinarily would not be included in the final model. That being said, sex may have some other clinical significance that we wouldn't ordinarily be aware of. Thus we elect to keep it in the model anyway. There is the potential of sex having other significant interactions, but since we aren't testing for that, we will ignore them for now.

### Model Use

The model is: $logit(\hat{\pi}) = −0.671 − 0.043S + 0.407R + 1.206T_1 + 0.968T_2$

For race <code>R</code>, we will assume <code>1 = white</code> and <code>0 = nonwhite</code>. For sex <code>S</code>, <code>1 = female</code> and <code>0 = male</code>. For drug type <code>T1</code> and <code>T2</code>, <code>T1</code> = 1, <code>T2</code> = 0 for alcohol, <code>T1</code> = 0, <code>T2</code> = 1 for cigarettes, <code>T1</code> = <code>T2</code> = 0 for marijuana.

To find the group with the highest probability of use of alcohol:<br>
$logit(\hat{\pi}) = −0.671 − 0.043S + 0.407R + 1.206$ is maximized when $S=0$ and $R=1$. So, the group with the highest probability of use of alochol is white males.

Given gender, the estimated odds of a white subject using a given substance are times the estimated odds for a nonwhite subject:<br>
White Subject: $−0.671 − 0.043S + 0.407(1) + 1.206T_1 + 0.968T_2$<br>
Non-white Subject: $−0.671 − 0.043S + 0.407(0) + 1.206T_1 + 0.968T_2$<br>
The difference between the two equations is the indicator for the $R$ value, so 0.407. Thus, the estimated odds of a white subject using a given substance is $e^{.407}=1.502$ times the estimated odds of a nonwhite subject using a given substance.